In [1]:
cd ..

D:\J.H.LEE\05. CODING\Python\Project\tdatlib


In [2]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:80% !important;}</style>"))

import plotly.graph_objects as go
import pandas as pd
# import tdatlib as tdat
from plotly.subplots import make_subplots
from tdatlib import tools, ecos, fred, krse, idex, kr

In [10]:
class bollinger_band(object):
    option = dict(key='TP', window=20, std=2)
    '''
    option
    ------
    key    : [str] 시가, 종가, 고가, 저가 또는 TP(Typical Price)
    window : [int] 이동 평균 개수
    std    : [int, float] 상/하한 표준 편차 범위 가중치
    '''
    def __init__(self, ohlcv:pd.DataFrame, name:str=str()):
        self.ohlcv = ohlcv
        self.name = name
        return
    
    @property
    def key(self) -> str:
        return self.option['key']
    
    @key.setter
    def key(self, key:str):
        self.option['key'] = key
    
    @property
    def window(self) -> int:
        return self.option['window']
    
    @window.setter
    def window(self, window:int):
        self.option['window'] = window
        
    @property
    def std(self) -> str:
        return self.option['std']
    
    @std.setter
    def std(self, std:int or float):
        self.option['std'] = std
    
    @property
    def basis(self) -> pd.Series:
        attr = f'__basis_{self.key}'
        if not hasattr(self, attr):
            _t = (self.ohlcv.고가 + self.ohlcv.저가 + self.ohlcv.종가) / 3 if self.key == 'TP' else self.ohlcv[self.key]
            self.__setattr__(attr, _t)
        return self.__getattribute__(attr)
    
    @property
    def middle(self) -> pd.Series:
        attr = f'__middle_{self.window}'
        if not hasattr(self, attr):
            _t = self.basis.rolling(window=self.window).mean()
            self.__setattr__(attr, _t)
        return self.__getattribute__(attr)
    
    @property
    def upper(self) -> pd.Series:
        attr = f'__upper_{self.std}'
        if not hasattr(self, attr):
            _t = self.middle + self.std * self.basis.rolling(window=self.window).std()
            self.__setattr__(attr, _t)
        return self.__getattribute__(attr)
    
    @property
    def lower(self) -> pd.Series:
        attr = f'__lower_{self.std}'
        if not hasattr(self, attr):
            _t = self.middle - self.std * self.basis.rolling(window=self.window).std()
            self.__setattr__(attr, _t)
        return self.__getattribute__(attr)

# The Basics


[ SUMMARY ]
* '80년대 John Bollinger에 의해 개발
* 중심선과 상/하 가격 밴드로 구성
* 중심선은 단순 이동 평균(SMA)으로 계산되며 상/하 밴드는 대상 증권의 표준 편차로 계산
* 밴드는 가격 변동이 클 때 팽창하고 작을 때 수축
* 상/하 밴드를 목표가로 트레이딩하기 위한 목적으로 개발

In [8]:
kosdaq = bollinger_band(ohlcv=idex.kosdaq, name='KOSDAQ')

In [9]:
kosdaq.basis

TypeError: hasattr expected 2 arguments, got 1